In [2]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tnrange


# Import Spike Trains

In [8]:
def is_float(string):
    """ True if given string is float else False"""
    try:
        return float(string)
    except ValueError:
        return False

data = []
with open("/media/hdd01/sklee/cont_shglm/CA1_inputs/Ispikes_d10_Ni200_Ri7.5_rseed1_rep2.dat", 'r') as f:
    d = f.readlines()
    for i in d:
        k = i.rstrip().split(" ")
        data.append([float(i) if is_float(i) else i for i in k]) 

data = np.array(data, dtype='O')

In [9]:
for i in range(data.shape[0]):
    if data[i,0] == "0":
        data[i,0] = 0

print(data.shape)
bad_idx = np.where(data[:,1] == "NA")
print(bad_idx[0].size)
data = np.delete(data, bad_idx[0], axis=0)

print(data.shape)
        
print(np.max(data[:,0]))
print(np.max(data[:,1]))

data = data[np.argsort(data[:,1])]

for i in range(data.shape[0]):
    data[i,0] = int(data[i,0])

(14005, 2)
0
(14005, 2)
199.0
10002.0


In [10]:
np.save("/media/hdd01/sklee/cont_shglm/CA1_inputs/Ispikes_d10_Ni200_Ri7.5_rseed1_rep2.npy", data)

# Make Neural Matrix

In [32]:
spike_train = np.load("/media/hdd01/sklee/cont_shglm/CA1_inputs/Ispikes_d10_Ni200_Ri7.5_rseed1_rep2.npy", allow_pickle=True)
print(spike_train.shape)

(14005, 2)


In [33]:
bin_size = 1 # in milliseconds
syn_no = int(np.max(spike_train[:,0])) + 1
bin_no = int(int(np.max(spike_train[:,1]) + 1) / bin_size)
neural = np.zeros((bin_no, syn_no))

bin_edges = np.arange(0, int(np.max(spike_train[:,1])) + 1 + bin_size, bin_size)
print(bin_edges.shape)
print(bin_edges)

hist_counts, hist_edges = np.histogram(spike_train[:,1], bin_edges)
print(np.sum(hist_counts))
print(spike_train.shape[0])
print(hist_counts.shape)

count = 0

for i in tnrange(hist_counts.shape[0]):
    bin_count = hist_counts[i]
    for j in range(bin_count):
        syn = spike_train[count,0]
        neural[i, syn] += 1
        count += 1
        
print(count)
print(np.sum(neural))
print(neural.shape)
print(np.nonzero(neural)[0].size/neural.size)
print(spike_train[-10:,])

(10004,)
[    0     1     2 ... 10001 10002 10003]
14005
14005
(10003,)


<ipython-input-33-3b51645672d0>:17: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  for i in tnrange(hist_counts.shape[0]):



14005
14005.0
(10003, 200)
0.007000399880035989
[[73 9997.0]
 [153 9997.0]
 [120 9998.0]
 [187 9998.0]
 [38 9999.0]
 [9 9999.0]
 [60 10000.0]
 [159 10001.0]
 [139 10001.0]
 [112 10002.0]]


In [35]:
np.save("/media/hdd01/sklee/cont_shglm/CA1_inputs/Ispikes_d10_Ni200_Ri7.5_rseed1_rep2_neural.npy", neural)

In [5]:
V = np.fromfile("/media/hdd01/sklee/cont_shglm/CA1_inputs/vdata_T10_Ne2000_gA0.6_tauA1_gN0.8_Ni200_gG0.7_gB1.2_Er1_Ir7.5_balanced_rep3_stimseed1.bin")
print(V.shape)
print(V)

(30004,)
[ 2.12220799e-310 -6.83000000e+001 -6.83000000e+001 ... -6.66190186e+001
 -6.80570024e+001 -6.50460029e+001]


In [16]:
one = np.load("/media/hdd01/sklee/cont_shglm/CA1_inputs/Ispikes_d10_Ni200_Ri7.5_rseed1_rep0_neural.npy")[:10001]
two = np.load("/media/hdd01/sklee/cont_shglm/CA1_inputs/Ispikes_d10_Ni200_Ri7.5_rseed1_rep1_neural.npy")[:10001]
three = np.load("/media/hdd01/sklee/cont_shglm/CA1_inputs/Ispikes_d10_Ni200_Ri7.5_rseed1_rep2_neural.npy")[:10001]

In [17]:
final = np.vstack((one, two, three))
print(final.shape)

(30003, 200)


In [18]:
np.save("/media/hdd01/sklee/cont_shglm/CA1_inputs/combined_S_I.npy", final)